In [1]:
import logging
from strategy.utils import read_orderbook_from_url
from strategy.strategy import StatisticalArbitrageBacktest

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

### First (basic) conditions

In [2]:
EXCHANGE_1 = "bitmex"
EXCHANGE_2 = "deribit"

LINKS = {
    EXCHANGE_1: "https://research-test.s3.eu-west-1.amazonaws.com/data/bitmex_XBTUSD_20230130_depth10.txt.gz",
    EXCHANGE_2: "https://research-test.s3.eu-west-1.amazonaws.com/data/deribit_BTC-PERPETUAL_20230130_depth10.txt.gz"
}

FEES = {EXCHANGE_1: 0.00005, EXCHANGE_2: 0.0001}
MAX_POSITION_USD = 1_000_000
LATENCY_MS = 5

Initialize strategy

In [3]:
arbitrage_backtest = StatisticalArbitrageBacktest(
    max_position_usd=MAX_POSITION_USD, fees=FEES, latency_ms=LATENCY_MS
)

Load orderbook data

In [4]:
bitmex_orderbook = read_orderbook_from_url(LINKS[EXCHANGE_1])
deribit_orderbook = read_orderbook_from_url(LINKS[EXCHANGE_2])

In [5]:
arbitrage_backtest.set_data(
    {
        EXCHANGE_1: bitmex_orderbook.sort_values("timestamp"),
        EXCHANGE_2: deribit_orderbook.sort_values("timestamp"),
    }
)

INFO:strategy.strategy:Setting orderbook data for bitmex, deribit...
INFO:strategy.strategy:Order book data successfully set


In [6]:
arbitrage_backtest.get_data()

,timestamp_bitmex,side,price_bitmex,volume_bitmex,timestamp_ms,timestamp_deribit,price_deribit,volume_deribit
0,2023-01-30 00:00:00.142054,Bid,23516.5,4900.0,1675036800142,2023-01-30 00:00:00.141285,23757.0,840.0
1,2023-01-30 00:00:00.143591,Bid,23736.5,11900.0,1675036800143,2023-01-30 00:00:00.141285,23757.0,840.0
2,2023-01-30 00:00:00.143591,Bid,23516.5,4900.0,1675036800143,2023-01-30 00:00:00.141285,23757.0,840.0
3,2023-01-30 00:00:00.143945,Ask,23764.5,300.0,1675036800143,2023-01-30 00:00:00.141285,23771.5,47540.0
4,2023-01-30 00:00:00.143945,Ask,23746.5,141700.0,1675036800143,2023-01-30 00:00:00.141285,23771.5,47540.0
...,...,...,...,...,...,...,...,...
120851267,2023-01-30 23:59:59.957913,Bid,22806.5,30700.0,1675123199957,2023-01-30 23:59:59.960547,22825.0,13880.0
120851268,2023-01-30 23:59:59.957913,Bid,22807.0,100.0,1675123199957,2023-01-30 23:59:59.960547,22825.0,13880.0
120851269,2023-01-30 23:59:59.957913,Ask,22808.5,33400.0,1675123199957,2023-01-30 23:59:59.960547,22835.5,54200.0
120851270,2023-01-30 23:59:59.957913,Ask,22808.0,82400.0,1675123199957,2023-01-30 23:59:59.960547,22835.5,54200.0


Run backtest

In [322]:
res = arbitrage_backtest.run()

INFO:strategy.strategy:Running Statistical Arbitrage strategy for exchanges bitmex and deribit...
INFO:strategy.strategy:Processing orderbook data in 121 batches...
INFO:strategy.strategy:Processed batch 1/121, Profit: 0.00000018 BTC
INFO:strategy.strategy:Processed batch 2/121, Profit: 0.00000019 BTC
INFO:strategy.strategy:Processed batch 3/121, Profit: 0.00000019 BTC
INFO:strategy.strategy:Processed batch 4/121, Profit: 0.00000019 BTC
INFO:strategy.strategy:Processed batch 5/121, Profit: 0.00000019 BTC
INFO:strategy.strategy:Processed batch 6/121, Profit: 0.00000019 BTC
INFO:strategy.strategy:Processed batch 7/121, Profit: 0.00000019 BTC
INFO:strategy.strategy:Processed batch 8/121, Profit: 0.00000019 BTC
INFO:strategy.strategy:Processed batch 9/121, Profit: 0.00000021 BTC
INFO:strategy.strategy:Processed batch 10/121, Profit: 0.00000021 BTC
INFO:strategy.strategy:Processed batch 11/121, Profit: 0.00000021 BTC
INFO:strategy.strategy:Processed batch 12/121, Profit: 0.00000021 BTC
INFO

In [318]:
print(f"Total PnL: {res[0]} BTC")
print(f"Total volume traded: {res[1]} BTC")

Total PnL: 0.003590184746938369 BTC
Total volume traded: 126.86024116696836 BTC


### Extension 1

Find a profitable strategy if bitmex taker fee is $0.0175\%$ and deribit taker fee is $0.0500\%$.

In [331]:
NEW_FEES = {EXCHANGE_1: 0.000175, EXCHANGE_2: 0.0005}

In [333]:
arbitrage_backtest.fees = NEW_FEES

In [334]:
arbitrage_backtest.fees

{'bitmex': 0.000175, 'deribit': 0.0005}

In [335]:
res_new_fees = arbitrage_backtest.run()

INFO:strategy.strategy:Running Statistical Arbitrage strategy for exchanges bitmex and deribit...


INFO:strategy.strategy:Processing orderbook data in 121 batches...
INFO:strategy.strategy:Processed batch 1/121, Profit: -0.00000031 BTC
INFO:strategy.strategy:Processed batch 2/121, Profit: -0.00000030 BTC
INFO:strategy.strategy:Processed batch 3/121, Profit: -0.00000030 BTC
INFO:strategy.strategy:Processed batch 4/121, Profit: -0.00000030 BTC
INFO:strategy.strategy:Processed batch 5/121, Profit: -0.00000029 BTC
INFO:strategy.strategy:Processed batch 6/121, Profit: -0.00000029 BTC
INFO:strategy.strategy:Processed batch 7/121, Profit: -0.00000029 BTC
INFO:strategy.strategy:Processed batch 8/121, Profit: -0.00000029 BTC
INFO:strategy.strategy:Processed batch 9/121, Profit: -0.00000027 BTC
INFO:strategy.strategy:Processed batch 10/121, Profit: -0.00000027 BTC
INFO:strategy.strategy:Processed batch 11/121, Profit: -0.00000027 BTC
INFO:strategy.strategy:Processed batch 12/121, Profit: -0.00000027 BTC
INFO:strategy.strategy:Processed batch 13/121, Profit: -0.00000027 BTC
INFO:strategy.strat

In [336]:
print(f"Total PnL for new fees: {res_new_fees[0]} BTC")
print(f"Total volume traded for new fees: {res_new_fees[1]} BTC")

Total PnL for new fees: 0.00043062859077335185 BTC
Total volume traded for new fees: 129.274378881727 BTC


The strategy remains profitable but shows significantly reduced profitability compared to when using basic fees.

### Extension 2.

The latency is 50 milliseconds

In [ ]:
NEW_LATENCY = 50

In [342]:
arbitrage_backtest.fees = FEES
arbitrage_backtest.latency_ms = NEW_LATENCY

In [343]:
arbitrage_backtest.fees

{'bitmex': 5e-05, 'deribit': 0.0001}

In [344]:
arbitrage_backtest.latency_ms

50

In [348]:
res_new_latency = arbitrage_backtest.run()

INFO:strategy.strategy:Running Statistical Arbitrage strategy for exchanges bitmex and deribit...
INFO:strategy.strategy:Processing orderbook data in 121 batches...
INFO:strategy.strategy:Processed batch 1/121, Profit: 0.00000018 BTC
INFO:strategy.strategy:Processed batch 2/121, Profit: 0.00000018 BTC
INFO:strategy.strategy:Processed batch 3/121, Profit: 0.00000018 BTC
INFO:strategy.strategy:Processed batch 4/121, Profit: 0.00000018 BTC
INFO:strategy.strategy:Processed batch 5/121, Profit: 0.00000019 BTC
INFO:strategy.strategy:Processed batch 6/121, Profit: 0.00000020 BTC
INFO:strategy.strategy:Processed batch 7/121, Profit: 0.00000020 BTC
INFO:strategy.strategy:Processed batch 8/121, Profit: 0.00000020 BTC
INFO:strategy.strategy:Processed batch 9/121, Profit: 0.00000021 BTC
INFO:strategy.strategy:Processed batch 10/121, Profit: 0.00000021 BTC
INFO:strategy.strategy:Processed batch 11/121, Profit: 0.00000021 BTC
INFO:strategy.strategy:Processed batch 12/121, Profit: 0.00000021 BTC
INFO

In [349]:
print(f"Total PnL for new fees: {res_new_latency[0]} BTC")
print(f"Total volume traded for new fees: {res_new_latency[1]} BTC")

Total PnL for new fees: 0.0004082502951782334 BTC
Total volume traded for new fees: 129.22533352864014 BTC


Once again, with a latency of 50 ms, the strategy remains profitable but is significantly less so compared to a latency of 5 ms.